In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
numberOfFiles = 0
directory = '/content/drive/My Drive/12th grade/Senior Project/Healthy'
for filename in os.listdir(directory):
  numberOfFiles +=1
print(numberOfFiles)


972


In [0]:
import librosa
import os
import numpy as np

max_pad_len = 7223


audio, sample_rate = librosa.load('/content/drive/My Drive/12th grade/Senior Project/Healthy/1-a_n.wav', res_type='kaiser_fast')
print(sample_rate)
print(audio) 
try:
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    print(mfccs.shape)
    
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    print(mfccs.shape)
    
except Exception as e:
    print("Error encountered while parsing file: ", file_name)


22050
[0.11425632 0.1663573  0.11771581 ... 0.04529907 0.07436734 0.        ]
(40, 84)
(40, 7223)


In [0]:
import numpy as np
max_pad_len = 7223

# def extract_features(file_name):
   
#     try:
#         audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
#        # print (audio)
#        # print (sample_rate) 
#         mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
#         mfccsscaled = np.mean(mfccs.T,axis=0)
        
#     except Exception as e:
#         print(e + "Error encountered while parsing file: ", file_name)
#         return None 
     
#     return mfccsscaled

def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        #print(sample_rate)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        #print(mfccs.shape)
        
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs

# Load various imports 

import pandas as pd
import os
import librosa

features = []

#start with Healthy:

directory = '/content/drive/My Drive/12th grade/Senior Project/Healthy'

for filename in os.listdir(directory):
    if filename.endswith(".wav"):
      data = extract_features(directory+ '/'+ filename)
      features.append([data, "healthy"])
      

#next go through Pathological:
directory = '/content/drive/My Drive/12th grade/Senior Project/Pathological'

for filename in os.listdir(directory):
    if filename.endswith(".wav"):
      data = extract_features(directory+ '/'+ filename)
      features.append([data, "pathological"])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','label'])

print('Finished feature extraction from ', len(featuresdf), 'files')

Finished feature extraction from  2019 files


In [0]:
print(featuresdf)

                                                feature         label
0     [[-116.22905654781799, -126.04590981234061, -1...       healthy
1     [[-126.12949431757471, -147.89815989125637, -1...       healthy
2     [[-133.94486644305735, -160.52965450893257, -1...       healthy
3     [[-62.70337969346285, -95.36462212165884, -146...       healthy
4     [[-153.2609159072828, -172.51277819587517, -19...       healthy
...                                                 ...           ...
2014  [[-97.22572812506833, -93.25961983463002, -100...  pathological
2015  [[-131.52875166701983, -145.0058851281211, -15...  pathological
2016  [[-192.38812952123013, -209.52246804700602, -2...  pathological
2017  [[-144.49387423230138, -165.5888310210561, -20...  pathological
2018  [[-192.01177126134857, -192.9680438781097, -19...  pathological

[2019 rows x 2 columns]


In [0]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
#print(X)
y = np.array(featuresdf.label.tolist())
#print(y)
# Encode the classification labels
le = LabelEncoder()
#print(le)
yy = to_categorical(le.fit_transform(y)) 
# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)
print(y_train)
print(x_train.shape)
print(y_test.shape)

Using TensorFlow backend.


NameError: ignored

In [0]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_rows = 40
num_columns = 7223
num_channels = 1

x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax'))

In [0]:
print (x_train.shape)

NameError: ignored

In [0]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [0]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 39, 7222, 16)      80        
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 19, 3611, 16)      0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 19, 3611, 16)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 18, 3610, 32)      2080      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 9, 1805, 32)       0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 9, 1805, 32)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 1804, 64)      

In [0]:
num_epochs = 100
num_batch_size = 32



model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test))

Train on 1615 samples, validate on 404 samples
Epoch 1/100
1615/1615 [==============================] - 152s 94ms/step - loss: 0.6949 - accuracy: 0.5108 - val_loss: 0.6899 - val_accuracy: 0.5767
Epoch 2/100
1615/1615 [==============================] - 151s 93ms/step - loss: 0.6847 - accuracy: 0.5548 - val_loss: 0.6851 - val_accuracy: 0.5891
Epoch 3/100
1615/1615 [==============================] - 155s 96ms/step - loss: 0.6679 - accuracy: 0.6012 - val_loss: 0.6646 - val_accuracy: 0.6312
Epoch 4/100
1615/1615 [==============================] - 151s 94ms/step - loss: 0.6440 - accuracy: 0.6495 - val_loss: 0.6514 - val_accuracy: 0.6411
Epoch 5/100
1615/1615 [==============================] - 151s 93ms/step - loss: 0.6484 - accuracy: 0.6359 - val_loss: 0.6562 - val_accuracy: 0.6312
Epoch 6/100
1615/1615 [==============================] - 151s 93ms/step - loss: 0.6280 - accuracy: 0.6718 - val_loss: 0.6363 - val_accuracy: 0.6733
Epoch 7/100
1615/1615 [==============================] - 152s 94m

In [0]:
model.save('/content/my_model.pb') 

In [0]:
from google.colab import files

files.download('my_model.pb')

In [0]:
!pip install -q pyyaml h5py

In [0]:
keras.Model.save(model, 'audio_processing_model1')

ValueError: ignored

In [0]:
files.download('my_model1.h5') 

In [0]:
!mkdir -p saved_model
model.save('saved_model/my_model') 

In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 39, 7222, 16)      80        
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 19, 3611, 16)      0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 19, 3611, 16)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 18, 3610, 32)      2080      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 9, 1805, 32)       0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 9, 1805, 32)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 1804, 64)      

In [0]:
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9783281683921814
Testing Accuracy:  0.6732673048973083


In [0]:
!python --version

Python 3.6.9
